In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
from sklearn.model_selection import cross_val_score,KFold ## for regression
from sklearn.model_selection import StratifiedKFold ## recommended for classification

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import seaborn as sns
from sklearn import tree
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, mean_squared_error, r2_score
from sklearn.utils import class_weight
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings("ignore")

## Import Data

In [4]:
df = pd.read_csv('data_final_with_scaled.csv', index_col = 0)
df['tract']=df['tract'].apply(lambda x: str(x).zfill(11))
df

,tract,year,id,growth,growth_per_sqft,mean_rent,mean_rent_per,total_units,region,state_code,...,county_male_population,county_female_population,county_0-19_pop,county_20-29_pop,county_30-39_pop,county_40-49_pop,county_50-59_pop,county_60-69_pop,county_70+_pop,employment_rate
0,01003010500,2016,10030105002016,3.479098,3.968254,627.083333,0.655000,68.0,s,1,...,2705.500000,2858.0,1416.500000,537.500000,647.500000,712.00,704.000000,761.500000,763.50,0.939210
1,01003010500,2017,10030105002017,4.598007,4.071247,655.916667,0.681667,68.0,s,1,...,2728.000000,2961.0,1493.000000,524.500000,580.500000,653.50,704.500000,897.000000,705.50,0.956169
2,01003010500,2018,10030105002018,-5.107356,-4.889976,622.416667,0.648333,68.0,s,1,...,2701.500000,2806.0,1313.000000,563.500000,530.500000,661.00,703.500000,867.500000,826.00,0.957600
3,01003010500,2019,10030105002019,1.017539,1.285347,628.750000,0.656667,68.0,s,1,...,2991.000000,2816.0,1383.000000,625.000000,586.000000,728.50,748.000000,921.000000,852.50,0.976297
4,01003010500,2020,10030105002020,5.990722,5.837563,666.416667,0.695000,68.0,s,1,...,2801.000000,2764.0,1288.500000,574.000000,531.500000,756.50,725.500000,887.250000,806.75,0.965224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82269,56037970905,2022,560379709052022,2.966507,2.973662,914.600000,1.010000,264.0,w,56,...,1851.833333,1830.0,1036.000000,458.166667,536.000000,423.00,484.833333,394.250000,267.50,0.945222
82270,56037971100,2022,560379711002022,-0.977167,-0.883117,672.200000,0.954000,59.0,w,56,...,1851.833333,1830.0,1036.000000,458.166667,536.000000,423.00,484.833333,394.250000,267.50,0.941926
82271,56039967802,2022,560399678022022,32.601265,32.566588,3094.100000,4.858000,189.5,w,56,...,3488.125000,2900.5,1407.708333,810.208333,1061.166667,813.25,841.666667,652.833333,364.00,0.966028
82272,56041975301,2022,560419753012022,19.080019,19.001189,828.400000,0.834000,120.0,w,56,...,3598.000000,3497.0,2181.250000,786.750000,1034.500000,808.25,1041.000000,529.750000,476.00,0.919329


In [5]:
def get_year_df(year):
    df_year = df[df['year'] == year].reset_index(drop=True)
    list_year = ['tract', 'year']
    for i in range(2, len(df_year.columns)):
        list_year.append(f'{df_year.columns[i]}_{year}')
    df_year.columns = list_year
    
    return df_year

In [6]:
df_2022 = get_year_df(2022)
df_2021 = get_year_df(2021)
df_2020 = get_year_df(2020)
df_2019 = get_year_df(2019)
df_2018 = get_year_df(2018)
df_2017 = get_year_df(2017)
df_2016 = get_year_df(2016)

# Florida

In [7]:
df_2022['region_2022'].unique()

array(['s', 'se', 'w', 'sc', 'f', 'sw'], dtype=object)

In [8]:
df_2022_FL = df_2022[df_2022['region_2022'] == 'f']
df_2021_FL = df_2021[df_2021['region_2021'] == 'f']
df_2020_FL = df_2020[df_2020['region_2020'] == 'f']
df_2019_FL = df_2019[df_2019['region_2019'] == 'f']
df_2018_FL = df_2018[df_2018['region_2018'] == 'f']
df_2017_FL = df_2017[df_2017['region_2017'] == 'f']
df_2016_FL = df_2016[df_2016['region_2016'] == 'f']

In [9]:
df_y_FL = df_2022_FL.loc[:, ['tract', 'mean_rent_per_2022']]
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
df_x_FL = df_2022_FL.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
df_x_FL.columns = xlist
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
df_x_2019_col = ['tract']
df_x_lag1_col = ['tract']
for i in df_x_year_col:
    df_x_2019_col.append(f'{i}_2019')
    df_x_lag1_col.append(f'{i}_n-3')
df_x_2019_FL = df_2019_FL[df_x_2019_col]
df_x_2019_FL.columns = df_x_lag1_col
df_x_2018_col = ['tract']
df_x_lag2_col = ['tract']
for i in df_x_year_col:
    df_x_2018_col.append(f'{i}_2018')
    df_x_lag2_col.append(f'{i}_n-4')
df_x_2018_FL = df_2018_FL[df_x_2018_col]
df_x_2018_FL.columns = df_x_lag2_col
df_x_2017_col = ['tract']
df_x_lag3_col = ['tract']
for i in df_x_year_col:
    df_x_2017_col.append(f'{i}_2017')
    df_x_lag3_col.append(f'{i}_n-5')
df_x_2017_FL = df_2017_FL[df_x_2017_col]
df_x_2017_FL.columns = df_x_lag3_col

In [10]:
final_data_FL = df_y_FL.merge(df_x_FL, how='left', on='tract')
final_data_FL = final_data_FL.merge(df_x_2019_FL, how='left', on='tract')
final_data_FL = final_data_FL.merge(df_x_2018_FL, how='left', on='tract')
final_data_FL = final_data_FL.merge(df_x_2017_FL, how='left', on='tract')
final_data_FL.dropna(inplace = True)

In [11]:
X_FL = final_data_FL.iloc[:, 2:]
y_FL = final_data_FL.iloc[:, 1]

In [12]:
kf10 = KFold(n_splits = 10, shuffle = True)

LR = LinearRegression()
cv = cross_val_score(LR, X_FL, y_FL, cv = kf10)
LR_r2 = cv.mean()
print(cv, 'mean: ', cv.mean())

[0.92826912 0.9016716  0.90180379 0.91889927 0.92261643 0.93847092
 0.91137979 0.92417324 0.89093934 0.96133522] mean:  0.9199558730811315


# SouthEast

In [13]:
df_2022_SE = df_2022[df_2022['region_2022'] == 'se']
df_2021_SE = df_2021[df_2021['region_2021'] == 'se']
df_2020_SE = df_2020[df_2020['region_2020'] == 'se']
df_2019_SE = df_2019[df_2019['region_2019'] == 'se']
df_2018_SE = df_2018[df_2018['region_2018'] == 'se']
df_2017_SE = df_2017[df_2017['region_2017'] == 'se']
df_2016_SE = df_2016[df_2016['region_2016'] == 'se']

In [14]:
df_y_SE = df_2022_SE.loc[:, ['tract', 'mean_rent_per_2022']]

In [15]:
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
df_x_SE = df_2022_SE.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
df_x_SE.columns = xlist

In [16]:
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
# 2019
df_x_2019_col = ['tract']
df_x_lag1_col = ['tract']
for i in df_x_year_col:
    df_x_2019_col.append(f'{i}_2019')
    df_x_lag1_col.append(f'{i}_n-3')
df_x_2019_SE = df_2019_SE[df_x_2019_col]
df_x_2019_SE.columns = df_x_lag1_col
# 2018
df_x_2018_col = ['tract']
df_x_lag2_col = ['tract']
for i in df_x_year_col:
    df_x_2018_col.append(f'{i}_2018')
    df_x_lag2_col.append(f'{i}_n-4')
df_x_2018_SE = df_2018_SE[df_x_2018_col]
df_x_2018_SE.columns = df_x_lag2_col
#2017
df_x_2017_col = ['tract']
df_x_lag3_col = ['tract']
for i in df_x_year_col:
    df_x_2017_col.append(f'{i}_2017')
    df_x_lag3_col.append(f'{i}_n-5')
df_x_2017_SE = df_2017_SE[df_x_2017_col]
df_x_2017_SE.columns = df_x_lag3_col

In [17]:
final_data_SE = df_y_SE.merge(df_x_SE, how='left', on='tract')
final_data_SE = final_data_SE.merge(df_x_2019_SE, how='left', on='tract')
final_data_SE = final_data_SE.merge(df_x_2018_SE, how='left', on='tract')
final_data_SE = final_data_SE.merge(df_x_2017_SE, how='left', on='tract')
final_data_SE.dropna(inplace = True)

In [18]:
X_SE = final_data_SE.iloc[:, 2:]
y_SE = final_data_SE.iloc[:, 1]

In [19]:
kf10 = KFold(n_splits = 10, shuffle = True)

LR = LinearRegression()
cv = cross_val_score(LR, X_SE, y_SE, cv = kf10)
LR_r2 = cv.mean()
print(cv, 'mean: ', cv.mean())

[0.89726436 0.90989449 0.91346591 0.93989427 0.94228022 0.92792431
 0.94089996 0.91321092 0.92738244 0.94658843] mean:  0.9258805310614046


# South

In [20]:
df_2022_S = df_2022[df_2022['region_2022'] == 's']
df_2021_S = df_2021[df_2021['region_2021'] == 's']
df_2020_S = df_2020[df_2020['region_2020'] == 's']
df_2019_S = df_2019[df_2019['region_2019'] == 's']
df_2018_S = df_2018[df_2018['region_2018'] == 's']
df_2017_S = df_2017[df_2017['region_2017'] == 's']
df_2016_S = df_2016[df_2016['region_2016'] == 's']

In [21]:
df_y_S = df_2022_S.loc[:, ['tract', 'mean_rent_per_2022']]

In [22]:
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
df_x_S = df_2022_S.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
df_x_S.columns = xlist

In [23]:
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
# 2019
df_x_2019_col = ['tract']
df_x_lag1_col = ['tract']
for i in df_x_year_col:
    df_x_2019_col.append(f'{i}_2019')
    df_x_lag1_col.append(f'{i}_n-3')
df_x_2019_S = df_2019_S[df_x_2019_col]
df_x_2019_S.columns = df_x_lag1_col
# 2018
df_x_2018_col = ['tract']
df_x_lag2_col = ['tract']
for i in df_x_year_col:
    df_x_2018_col.append(f'{i}_2018')
    df_x_lag2_col.append(f'{i}_n-4')
df_x_2018_S = df_2018_S[df_x_2018_col]
df_x_2018_S.columns = df_x_lag2_col
#2017
df_x_2017_col = ['tract']
df_x_lag3_col = ['tract']
for i in df_x_year_col:
    df_x_2017_col.append(f'{i}_2017')
    df_x_lag3_col.append(f'{i}_n-5')
df_x_2017_S = df_2017_S[df_x_2017_col]
df_x_2017_S.columns = df_x_lag3_col

In [24]:
final_data_S = df_y_S.merge(df_x_S, how='left', on='tract')
final_data_S = final_data_S.merge(df_x_2019_S, how='left', on='tract')
final_data_S = final_data_S.merge(df_x_2018_S, how='left', on='tract')
final_data_S = final_data_S.merge(df_x_2017_S, how='left', on='tract')
final_data_S.dropna(inplace = True)

In [25]:
X_S = final_data_S.iloc[:, 2:]
y_S = final_data_S.iloc[:, 1]

In [26]:
kf10 = KFold(n_splits = 10, shuffle = True)

LR = LinearRegression()
cv = cross_val_score(LR, X_S, y_S, cv = kf10)
LR_r2 = cv.mean()
print(cv, 'mean: ', cv.mean())

[0.92259407 0.94373792 0.93315214 0.90722924 0.93732425 0.94614773
 0.90935393 0.93040918 0.91804593 0.95478903] mean:  0.9302783438469145


# SouthWest

In [27]:
df_2022_SW = df_2022[df_2022['region_2022'] == 'sw']
df_2021_SW = df_2021[df_2021['region_2021'] == 'sw']
df_2020_SW = df_2020[df_2020['region_2020'] == 'sw']
df_2019_SW = df_2019[df_2019['region_2019'] == 'sw']
df_2018_SW = df_2018[df_2018['region_2018'] == 'sw']
df_2017_SW = df_2017[df_2017['region_2017'] == 'sw']
df_2016_SW = df_2016[df_2016['region_2016'] == 'sw']

In [28]:
df_y_SW = df_2022_SW.loc[:, ['tract', 'mean_rent_per_2022']]

In [29]:
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
df_x_SW = df_2022_SW.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
df_x_SW.columns = xlist

In [30]:
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
# 2019
df_x_2019_col = ['tract']
df_x_lag1_col = ['tract']
for i in df_x_year_col:
    df_x_2019_col.append(f'{i}_2019')
    df_x_lag1_col.append(f'{i}_n-3')
df_x_2019_SW = df_2019_SW[df_x_2019_col]
df_x_2019_SW.columns = df_x_lag1_col
# 2018
df_x_2018_col = ['tract']
df_x_lag2_col = ['tract']
for i in df_x_year_col:
    df_x_2018_col.append(f'{i}_2018')
    df_x_lag2_col.append(f'{i}_n-4')
df_x_2018_SW = df_2018_SW[df_x_2018_col]
df_x_2018_SW.columns = df_x_lag2_col
#2017
df_x_2017_col = ['tract']
df_x_lag3_col = ['tract']
for i in df_x_year_col:
    df_x_2017_col.append(f'{i}_2017')
    df_x_lag3_col.append(f'{i}_n-5')
df_x_2017_SW = df_2017_SW[df_x_2017_col]
df_x_2017_SW.columns = df_x_lag3_col

In [31]:
final_data_SW = df_y_SW.merge(df_x_SW, how='left', on='tract')
final_data_SW = final_data_SW.merge(df_x_2019_SW, how='left', on='tract')
final_data_SW = final_data_SW.merge(df_x_2018_SW, how='left', on='tract')
final_data_SW = final_data_SW.merge(df_x_2017_SW, how='left', on='tract')
final_data_SW.dropna(inplace = True)

In [32]:
X_SW = final_data_SW.iloc[:, 2:]
y_SW = final_data_SW.iloc[:, 1]

In [33]:
kf10 = KFold(n_splits = 10, shuffle = True)

LR = LinearRegression()
cv = cross_val_score(LR, X_SW, y_SW, cv = kf10)
LR_r2 = cv.mean()
print(cv, 'mean: ', cv.mean())

[0.92784781 0.91942557 0.91826354 0.92424671 0.92917734 0.92231698
 0.90692643 0.9108675  0.90918102 0.93772166] mean:  0.9205974565402867


# Western

In [34]:
df_2022_W = df_2022[df_2022['region_2022'] == 'w']
df_2021_W = df_2021[df_2021['region_2021'] == 'w']
df_2020_W = df_2020[df_2020['region_2020'] == 'w']
df_2019_W = df_2019[df_2019['region_2019'] == 'w']
df_2018_W = df_2018[df_2018['region_2018'] == 'w']
df_2017_W = df_2017[df_2017['region_2017'] == 'w']
df_2016_W = df_2016[df_2016['region_2016'] == 'w']

In [35]:
df_y_W = df_2022_W.loc[:, ['tract', 'mean_rent_per_2022']]

In [36]:
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
df_x_W = df_2022_W.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
df_x_W.columns = xlist

In [37]:
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
# 2019
df_x_2019_col = ['tract']
df_x_lag1_col = ['tract']
for i in df_x_year_col:
    df_x_2019_col.append(f'{i}_2019')
    df_x_lag1_col.append(f'{i}_n-3')
df_x_2019_W = df_2019_W[df_x_2019_col]
df_x_2019_W.columns = df_x_lag1_col
# 2018
df_x_2018_col = ['tract']
df_x_lag2_col = ['tract']
for i in df_x_year_col:
    df_x_2018_col.append(f'{i}_2018')
    df_x_lag2_col.append(f'{i}_n-4')
df_x_2018_W = df_2018_W[df_x_2018_col]
df_x_2018_W.columns = df_x_lag2_col
#2017
df_x_2017_col = ['tract']
df_x_lag3_col = ['tract']
for i in df_x_year_col:
    df_x_2017_col.append(f'{i}_2017')
    df_x_lag3_col.append(f'{i}_n-5')
df_x_2017_W = df_2017_W[df_x_2017_col]
df_x_2017_W.columns = df_x_lag3_col

In [38]:
final_data_W = df_y_W.merge(df_x_W, how='left', on='tract')
final_data_W = final_data_W.merge(df_x_2019_W, how='left', on='tract')
final_data_W = final_data_W.merge(df_x_2018_W, how='left', on='tract')
final_data_W = final_data_W.merge(df_x_2017_W, how='left', on='tract')
final_data_W.dropna(inplace = True)

In [39]:
X_W = final_data_W.iloc[:, 2:]
y_W = final_data_W.iloc[:, 1]

In [40]:
kf10 = KFold(n_splits = 10, shuffle = True)

LR = LinearRegression()
cv = cross_val_score(LR, X_W, y_W, cv = kf10)
LR_r2 = cv.mean()
print(cv, 'mean: ', cv.mean())

[0.83038411 0.88510744 0.88048269 0.86652583 0.83014476 0.88353266
 0.90697818 0.80936344 0.90226636 0.88368274] mean:  0.8678468209675051


# South California

In [41]:
df_2022_SC = df_2022[df_2022['region_2022'] == 'sc']
df_2021_SC = df_2021[df_2021['region_2021'] == 'sc']
df_2020_SC = df_2020[df_2020['region_2020'] == 'sc']
df_2019_SC = df_2019[df_2019['region_2019'] == 'sc']
df_2018_SC = df_2018[df_2018['region_2018'] == 'sc']
df_2017_SC = df_2017[df_2017['region_2017'] == 'sc']
df_2016_SC = df_2016[df_2016['region_2016'] == 'sc']

In [42]:
df_y_SC = df_2022_SC.loc[:, ['tract', 'mean_rent_per_2022']]

In [43]:
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
df_x_SC = df_2022_SC.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
df_x_SC.columns = xlist

In [44]:
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
# 2019
df_x_2019_col = ['tract']
df_x_lag1_col = ['tract']
for i in df_x_year_col:
    df_x_2019_col.append(f'{i}_2019')
    df_x_lag1_col.append(f'{i}_n-3')
df_x_2019_SC = df_2019_SC[df_x_2019_col]
df_x_2019_SC.columns = df_x_lag1_col
# 2018
df_x_2018_col = ['tract']
df_x_lag2_col = ['tract']
for i in df_x_year_col:
    df_x_2018_col.append(f'{i}_2018')
    df_x_lag2_col.append(f'{i}_n-4')
df_x_2018_SC = df_2018_SC[df_x_2018_col]
df_x_2018_SC.columns = df_x_lag2_col
#2017
df_x_2017_col = ['tract']
df_x_lag3_col = ['tract']
for i in df_x_year_col:
    df_x_2017_col.append(f'{i}_2017')
    df_x_lag3_col.append(f'{i}_n-5')
df_x_2017_SC = df_2017_SC[df_x_2017_col]
df_x_2017_SC.columns = df_x_lag3_col

In [45]:
final_data_SC = df_y_SC.merge(df_x_SC, how='left', on='tract')
final_data_SC = final_data_SC.merge(df_x_2019_SC, how='left', on='tract')
final_data_SC = final_data_SC.merge(df_x_2018_SC, how='left', on='tract')
final_data_SC = final_data_SC.merge(df_x_2017_SC, how='left', on='tract')
final_data_SC.dropna(inplace = True)

In [46]:
X_SC = final_data_SC.iloc[:, 2:]
y_SC = final_data_SC.iloc[:, 1]

In [47]:
kf10 = KFold(n_splits = 10, shuffle = True)

LR = LinearRegression()
cv = cross_val_score(LR, X_SC, y_SC, cv = kf10)
LR_r2 = cv.mean()
print(cv, 'mean: ', cv.mean())

[0.8913496  0.90272336 0.89487395 0.84106659 0.84313668 0.87301829
 0.90444752 0.89931726 0.90835931 0.83593179] mean:  0.8794224354727855


# Whole Region

## Train the model: Use 2017-2019 to predict 2022

In [48]:
# y variable
df_y = df_2022.loc[:, ['tract', 'mean_rent_per_2022']]

In [49]:
# cross-sectional x variables
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
df_x = df_2022.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
df_x.columns = xlist
df_x

,tract,total_units,property_sqft,current_rent,income_tax_rate,corporate_tax_rate,state_sales_tax_rate,avg_local_sales_tax_rate,max_local_sales_tax,avg_household_income,median_debt,starbucks_vists_per,grocery_vists_per,hospitals_per,colleges_per,banks_per,public_schools_per,private_schools_per
0,01003010500,68.000000,65200.000000,633.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.000000,0.002987,0.002987,0.002987,0.000000,0.017921,0.000000
1,01003010600,56.000000,61800.000000,691.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.000000,0.002987,0.002987,0.002987,0.000000,0.017921,0.000000
2,01003010704,180.000000,160288.000000,1417.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.000000,0.026738,0.026738,0.026738,0.000000,0.133690,0.053476
3,01003010707,216.000000,207324.000000,1419.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.048626,0.048626,0.000000,0.000000,0.000000,0.097253,0.000000
4,01003010708,228.666667,269810.333333,1395.333333,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.048626,0.048626,0.000000,0.000000,0.000000,0.097253,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12460,56037970905,264.000000,239160.000000,965.000000,0.000000,0.000,0.04,0.012235,0.020,84961.0,2466.0,0.000720,0.001440,0.001440,0.000720,0.001440,0.007922,0.000720
12461,56037971100,59.000000,41568.000000,656.000000,0.000000,0.000,0.04,0.012235,0.020,84961.0,2466.0,0.000720,0.001440,0.001440,0.000720,0.001440,0.007922,0.000720
12462,56039967802,189.500000,144602.000000,2352.000000,0.000000,0.000,0.04,0.012235,0.020,128868.0,944.0,0.003714,0.002476,0.001238,0.000000,0.003714,0.007427,0.002476
12463,56041975301,120.000000,119076.000000,739.000000,0.000000,0.000,0.04,0.012235,0.020,71005.0,1877.0,0.000000,0.000875,0.000875,0.000000,0.000000,0.000000,0.000000


In [50]:
# time-series x variables
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
df_x_2019_col = ['tract']
df_x_lag1_col = ['tract']
for i in df_x_year_col:
    df_x_2019_col.append(f'{i}_2019')
    df_x_lag1_col.append(f'{i}_n-3')
df_x_2019 = df_2019[df_x_2019_col]
df_x_2019.columns = df_x_lag1_col
df_x_2019

,tract,mean_rent_per_n-3,median_income_n-3,population_n-3,white_pop_n-3,black_pop_n-3,native_pop_n-3,asian_pop_n-3,pacific_pop_n-3,male_population_n-3,...,stocks_bonds_n-3,stocks_corp_bonds_n-3,hist_risk_n-3,inflation_n-3,SP500_n-3,Tbill_3month_n-3,Tbond_10year_n-3,baa_corp_bonds_n-3,real_estate3_n-3,employment_rate_n-3
0,01003010500,0.656667,60208.5,5463.0,4689.0,669.0,19.0,8.0,0.0,3001.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.976297
1,01003010600,0.592500,32973.0,3730.0,1209.0,2335.0,7.0,0.0,0.0,1846.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.934051
2,01003010704,1.045833,74283.0,6456.0,4287.0,2051.0,0.0,63.0,0.0,2925.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.972301
3,01003010707,1.095000,54684.0,5935.0,4899.5,335.0,17.5,11.0,0.0,2991.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.916468
4,01003010708,1.007500,54684.0,5935.0,4899.5,335.0,17.5,11.0,0.0,2991.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.916468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11762,49057210518,1.155417,66944.5,4559.0,4158.0,28.0,19.0,33.0,0.0,2316.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.950816
11763,49057210600,0.923333,59563.0,7670.0,6605.0,28.0,18.0,90.0,62.0,3950.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.938847
11764,49057210701,1.123333,55643.5,6306.0,5236.0,14.0,95.0,73.0,0.0,3110.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.937280
11765,49057211100,0.916111,71190.0,5526.0,5157.0,15.0,0.0,19.0,0.0,2519.0,...,0.2158,0.1588,0.0483,0.0229,0.2828,-0.0072,0.0719,0.1275,0.0138,0.956245


In [51]:
df_x_2018_col = ['tract']
df_x_lag2_col = ['tract']
for i in df_x_year_col:
    df_x_2018_col.append(f'{i}_2018')
    df_x_lag2_col.append(f'{i}_n-4')
df_x_2018 = df_2018.loc[:, df_x_2018_col]
df_x_2018.columns = df_x_lag2_col
df_x_2018

,tract,mean_rent_per_n-4,median_income_n-4,population_n-4,white_pop_n-4,black_pop_n-4,native_pop_n-4,asian_pop_n-4,pacific_pop_n-4,male_population_n-4,...,stocks_bonds_n-4,stocks_corp_bonds_n-4,hist_risk_n-4,inflation_n-4,SP500_n-4,Tbill_3month_n-4,Tbond_10year_n-4,baa_corp_bonds_n-4,real_estate3_n-4,employment_rate_n-4
0,01003010500,0.648333,51884.5,5444.0,4703.0,585.0,13.0,40.0,0.0,2927.0,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.957600
1,01003010600,0.594167,28739.0,3565.0,1139.0,2268.0,0.0,0.0,0.0,1761.0,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.895397
2,01003010704,0.965000,73590.5,5880.0,4577.0,1150.0,0.0,51.0,0.0,2745.0,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.962751
3,01003010707,0.982500,54163.5,5673.5,4768.5,364.5,15.5,13.0,0.0,2701.5,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.946300
4,01003010708,0.985000,54163.5,5673.5,4768.5,364.5,15.5,13.0,0.0,2701.5,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.946300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11623,49057210514,1.140000,63555.5,4500.0,4025.0,34.5,16.0,29.0,0.0,2266.5,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.937159
11624,49057210600,0.879167,59605.0,7579.0,6484.0,30.0,42.0,179.0,35.0,3940.0,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.957966
11625,49057210701,1.090833,56976.0,6190.0,5023.0,12.0,95.0,59.0,0.0,3153.0,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.911193
11626,49057211100,0.853056,53981.0,5367.0,4946.0,7.0,0.0,21.0,0.0,2368.0,...,-0.0421,-0.0146,0.0466,0.0191,-0.0602,0.0003,-0.0189,-0.0459,0.0257,0.952327


In [52]:
df_x_2017_col = ['tract']
df_x_lag3_col = ['tract']
for i in df_x_year_col:
    df_x_2017_col.append(f'{i}_2017')
    df_x_lag3_col.append(f'{i}_n-5')
df_x_2017 = df_2017.loc[:, df_x_2017_col]
df_x_2017.columns = df_x_lag3_col
df_x_2017

,tract,mean_rent_per_n-5,median_income_n-5,population_n-5,white_pop_n-5,black_pop_n-5,native_pop_n-5,asian_pop_n-5,pacific_pop_n-5,male_population_n-5,...,stocks_bonds_n-5,stocks_corp_bonds_n-5,hist_risk_n-5,inflation_n-5,SP500_n-5,Tbill_3month_n-5,Tbond_10year_n-5,baa_corp_bonds_n-5,real_estate3_n-5,employment_rate_n-5
0,01003010500,0.681667,39538.0,5038.0,4191.0,735.0,17.0,36.0,0.0,2750.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.956169
1,01003010600,0.582500,14911.0,3418.0,913.0,2373.0,0.0,0.0,0.0,1682.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.940697
2,01003010704,0.915000,72317.0,5883.0,4729.0,1032.0,0.0,44.0,0.0,2799.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.939986
3,01003010707,0.924167,49744.5,5580.5,4875.0,372.5,22.0,10.5,0.0,2728.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.931988
4,01003010708,0.998611,49744.5,5580.5,4875.0,372.5,22.0,10.5,0.0,2728.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.931988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11337,49057210514,1.032500,62724.0,4530.5,3930.0,34.5,21.0,28.0,0.0,2262.5,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.934263
11338,49057210600,0.803333,62948.0,7360.0,6559.0,0.0,24.0,151.0,16.0,3787.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.933432
11339,49057210701,1.003333,52810.0,5945.0,4767.0,31.0,74.0,47.0,0.0,3056.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.895544
11340,49057211100,0.809167,54231.0,5302.0,4805.0,26.0,0.0,10.0,0.0,2429.0,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.958148


In [53]:
final_data = df_y.merge(df_x, how='left', on='tract')
final_data = final_data.merge(df_x_2019, how='left', on='tract')
final_data = final_data.merge(df_x_2018, how='left', on='tract')
final_data = final_data.merge(df_x_2017, how='left', on='tract')
final_data.dropna(inplace = True)
final_data.shape

(11340, 136)

In [54]:
final_data.head()

,tract,mean_rent_per_2022,total_units,property_sqft,current_rent,income_tax_rate,corporate_tax_rate,state_sales_tax_rate,avg_local_sales_tax_rate,max_local_sales_tax,...,stocks_bonds_n-5,stocks_corp_bonds_n-5,hist_risk_n-5,inflation_n-5,SP500_n-5,Tbill_3month_n-5,Tbond_10year_n-5,baa_corp_bonds_n-5,real_estate3_n-5,employment_rate_n-5
0,01003010500,0.694000,68.000000,65200.000000,633.000000,0.036667,0.065,0.04,0.052386,0.075,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.956169
1,01003010600,0.670000,56.000000,61800.000000,691.000000,0.036667,0.065,0.04,0.052386,0.075,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.940697
2,01003010704,1.424000,180.000000,160288.000000,1417.000000,0.036667,0.065,0.04,0.052386,0.075,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.939986
3,01003010707,1.428000,216.000000,207324.000000,1419.000000,0.036667,0.065,0.04,0.052386,0.075,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.931988
4,01003010708,1.213333,228.666667,269810.333333,1395.333333,0.036667,0.065,0.04,0.052386,0.075,...,0.188,0.1188,0.0477,0.0211,0.1909,-0.0115,0.0068,0.0746,0.0402,0.931988


In [55]:
X = final_data.iloc[:, 2:]
y = final_data.iloc[:, 1]

## GridSearchCV

In [56]:
# # Linear Regression
# kf10 = KFold(n_splits = 10, shuffle = True)

# LR = LinearRegression()
# cv = cross_val_score(LR, X, y, cv = kf10)
# LR_r2 = cv.mean()
# print(cv, 'mean: ', cv.mean())

In [57]:
# # Decision Tree
# kf10 = KFold(n_splits = 10, shuffle = True)
# DT_param = {'splitter' : ['best','random'],
#              'max_depth': [2, 4, 5, 10, None],
#              'min_samples_split'  : [2,5,10,20,50],
#              'min_samples_leaf'  : [1,5,10]}

# DT = DecisionTreeRegressor()
# DT_cv = GridSearchCV(DT, DT_param, cv = kf10, refit=True, verbose=3)
# DT_cv.fit(X, y)
# DT_r2 = DT_cv.best_score_
# DT_best_parm = DT_cv.best_params_
# print(DT_cv.best_score_)
# print(DT_cv.best_params_)

In [58]:
# # Random Forest
# kf10 = KFold(n_splits = 10, shuffle = True)
# RF_param = {'n_estimators' : [100, 150, 200],
#              'max_depth': [2, 4, 5, 10],
#              'min_samples_split'  : [2,5,10,20],
#              'min_samples_leaf'  : [1,5,10]}

# RF = RandomForestRegressor()
# RF_cv = GridSearchCV(RF, RF_param, cv = kf10, refit=True, verbose=3)
# RF_cv.fit(X, y)
# RF_r2 = RF_cv.best_score_
# RF_best_parm = RF_cv.best_params_
# print(RF_cv.best_score_)
# print(RF_cv.best_params_)

In [59]:
# # Xgboost
# kf10 = KFold(n_splits = 10, shuffle = True)
# XG_param = {'n_estimators' : [100, 150, 200, 500, 800], 
#             'learning_rate' : [0.1, 0.01],
#              'max_depth': [2, 4, 5, 10, None]}

# XG = XGBRegressor()
# XG_cv = GridSearchCV(XG, XG_param, cv = kf10, refit=True, verbose=3)
# XG_cv.fit(X, y)
# XG_r2 = XG_cv.best_score_
# XG_best_parm = XG_cv.best_params_
# print(XG_cv.best_score_)
# print(XG_cv.best_params_)

In [60]:
# # LGB
# kf10 = KFold(n_splits = 10, shuffle = True)
# LG_param = {'n_estimators' : [100, 150, 200, 500], 
#             'learning_rate' : [0.1, 0.01],
#              'max_depth': [2, 4, 5, 10, -1]}

# LG = LGBMRegressor()
# LG_cv = GridSearchCV(LG, LG_param, cv = kf10, refit=True, verbose=3)
# LG_cv.fit(X, y)
# LG_r2 = LG_cv.best_score_
# LG_best_parm = LG_cv.best_params_
# print(LG_cv.best_score_)
# print(LG_cv.best_params_)

## 2025 Prediction

In [61]:
xlist = ['tract', 'total_units', 'property_sqft', 'current_rent', 'income_tax_rate','corporate_tax_rate', 
       'state_sales_tax_rate', 'avg_local_sales_tax_rate', 'max_local_sales_tax', 'avg_household_income', 
       'median_debt', 'starbucks_vists_per', 'grocery_vists_per', 'hospitals_per', 'colleges_per', 
       'banks_per', 'public_schools_per', 'private_schools_per']
test_x = df_2022.loc[:, ['tract', 'total_units_2022', 'property_sqft_2022', 'current_rent_2022',
       'income_tax_rate_2022','corporate_tax_rate_2022', 'state_sales_tax_rate_2022', 
       'avg_local_sales_tax_rate_2022', 'max_local_sales_tax_2022', 'avg_household_income_2022', 
       'median_debt_2022', 'starbucks_vists_per_2022', 'grocery_vists_per_2022', 'hospitals_per_2022', 
       'colleges_per_2022', 'banks_per_2022', 'public_schools_per_2022', 'private_schools_per_2022']]
test_x.columns = xlist
test_x.head()

,tract,total_units,property_sqft,current_rent,income_tax_rate,corporate_tax_rate,state_sales_tax_rate,avg_local_sales_tax_rate,max_local_sales_tax,avg_household_income,median_debt,starbucks_vists_per,grocery_vists_per,hospitals_per,colleges_per,banks_per,public_schools_per,private_schools_per
0,01003010500,68.000000,65200.000000,633.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.000000,0.002987,0.002987,0.002987,0.0,0.017921,0.000000
1,01003010600,56.000000,61800.000000,691.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.000000,0.002987,0.002987,0.002987,0.0,0.017921,0.000000
2,01003010704,180.000000,160288.000000,1417.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.000000,0.026738,0.026738,0.026738,0.0,0.133690,0.053476
3,01003010707,216.000000,207324.000000,1419.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.048626,0.048626,0.000000,0.000000,0.0,0.097253,0.000000
4,01003010708,228.666667,269810.333333,1395.333333,0.036667,0.065,0.04,0.052386,0.075,80251.0,2219.0,0.048626,0.048626,0.000000,0.000000,0.0,0.097253,0.000000


In [62]:
df_x_year_col = ['mean_rent_per', 'median_income', 'population', 'white_pop', 'black_pop', 
        'native_pop', 'asian_pop', 'pacific_pop', 'male_population', 'pop_0_19', 'pop_20_29', 'pop_30_39', 
        'pop_40_49', 'pop_50_59', 'pop_60_69', 'occupancy_rate', 'house_supply', 'crime_rate', 'ROI_SP500', 
        'ROI_Tbill', 'ROI_Tbond', 'corp_bond', 'real_estate', 'sp500', 'Tbill', 'Tbond5', 'baa_corp_bond', 
        'real_estate2', 'stocks_bills', 'stocks_bonds', 'stocks_corp_bonds', 'hist_risk', 'inflation', 'SP500', 
        'Tbill_3month', 'Tbond_10year', 'baa_corp_bonds', 'real_estate3', 'employment_rate']
test_x_2022_col = ['tract']
test_x_lag1_col = ['tract']
for i in df_x_year_col:
    test_x_2022_col.append(f'{i}_2022')
    test_x_lag1_col.append(f'{i}_n-3')
test_x_2022 = df_2022[test_x_2022_col]
test_x_2022.columns = test_x_lag1_col
test_x_2022

,tract,mean_rent_per_n-3,median_income_n-3,population_n-3,white_pop_n-3,black_pop_n-3,native_pop_n-3,asian_pop_n-3,pacific_pop_n-3,male_population_n-3,...,stocks_bonds_n-3,stocks_corp_bonds_n-3,hist_risk_n-3,inflation_n-3,SP500_n-3,Tbill_3month_n-3,Tbond_10year_n-3,baa_corp_bonds_n-3,real_estate3_n-3,employment_rate_n-3
0,01003010500,0.694000,31352.833333,4298.333333,2389.666667,1874.666667,5.333333,2.000000,0.0,2308.333333,...,0.2198,0.14475,0.04875,0.0444,0.1489,-0.0402,-0.05935,0.0131,0.06275,0.948902
1,01003010600,0.670000,57806.333333,6663.666667,5371.666667,957.000000,15.666667,33.750000,0.0,3325.666667,...,0.2198,0.14475,0.04875,0.0444,0.1489,-0.0402,-0.05935,0.0131,0.06275,0.946879
2,01003010704,1.424000,58369.666667,8325.333333,6564.333333,1433.333333,0.000000,80.333333,0.0,4121.000000,...,0.2198,0.14475,0.04875,0.0444,0.1489,-0.0402,-0.05935,0.0131,0.06275,0.940757
3,01003010707,1.428000,52166.333333,5941.875000,5342.500000,476.750000,21.375000,24.375000,0.0,2820.791667,...,0.2198,0.14475,0.04875,0.0444,0.1489,-0.0402,-0.05935,0.0131,0.06275,0.950893
4,01003010708,1.213333,52166.333333,5941.875000,5342.500000,476.750000,21.375000,24.375000,0.0,2820.791667,...,0.3288,0.27540,0.05130,0.0681,0.2028,-0.0632,-0.10510,-0.0550,0.09380,0.950893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12460,56037970905,1.010000,70273.583333,3719.000000,3363.000000,17.875000,14.333333,14.916667,0.0,1851.833333,...,0.3288,0.27540,0.05130,0.0681,0.2028,-0.0632,-0.10510,-0.0550,0.09380,0.945222
12461,56037971100,0.954000,80257.000000,3154.666667,2834.000000,176.000000,4.333333,13.333333,0.0,1664.666667,...,0.3288,0.27540,0.05130,0.0681,0.2028,-0.0632,-0.10510,-0.0550,0.09380,0.941926
12462,56039967802,4.858000,77756.520833,6471.875000,6038.666667,51.625000,9.875000,39.125000,0.0,3488.125000,...,0.3288,0.27540,0.05130,0.0681,0.2028,-0.0632,-0.10510,-0.0550,0.09380,0.966028
12463,56041975301,0.834000,51420.000000,7170.750000,6709.750000,10.750000,37.250000,4.750000,0.0,3598.000000,...,0.3288,0.27540,0.05130,0.0681,0.2028,-0.0632,-0.10510,-0.0550,0.09380,0.919329


In [63]:
test_x_2021_col = ['tract']
test_x_lag2_col = ['tract']
for i in df_x_year_col:
    test_x_2021_col.append(f'{i}_2021')
    test_x_lag2_col.append(f'{i}_n-4')
test_x_2021 = df_2021.loc[:, test_x_2021_col]
test_x_2021.columns = test_x_lag2_col
test_x_2021

,tract,mean_rent_per_n-4,median_income_n-4,population_n-4,white_pop_n-4,black_pop_n-4,native_pop_n-4,asian_pop_n-4,pacific_pop_n-4,male_population_n-4,...,stocks_bonds_n-4,stocks_corp_bonds_n-4,hist_risk_n-4,inflation_n-4,SP500_n-4,Tbill_3month_n-4,Tbond_10year_n-4,baa_corp_bonds_n-4,real_estate3_n-4,employment_rate_n-4
0,01003010500,0.676667,46292.666667,5093.666667,3705.333333,1320.333333,10.666667,4.000000,0.000000,2761.666667,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.958614
1,01003010600,0.624167,44903.166667,5389.333333,3424.333333,1648.000000,17.333333,22.500000,0.000000,2502.333333,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.907272
2,01003010704,1.367500,69083.333333,7215.666667,5844.666667,1094.666667,0.000000,61.666667,0.000000,3520.000000,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.958504
3,01003010707,1.402500,52134.666667,5442.500000,4765.833333,516.333333,18.166667,24.750000,0.000000,2777.166667,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.948586
4,01003010708,1.094167,52134.666667,5442.500000,4765.833333,516.333333,18.166667,24.750000,0.000000,2777.166667,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.948586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12163,49057210518,1.307500,64327.083333,4323.500000,3633.333333,43.083333,29.000000,45.000000,6.750000,2206.666667,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.939983
12164,49057210600,1.125833,64205.833333,6908.333333,6079.666667,23.333333,43.666667,55.666667,39.333333,3556.000000,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.938740
12165,49057210701,1.339583,63055.666667,4835.333333,4144.000000,24.000000,31.666667,51.666667,0.000000,2401.333333,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.965871
12166,49057211100,1.104444,74185.000000,5264.000000,4728.666667,56.666667,0.000000,23.000000,0.000000,2512.666667,...,0.3288,0.2754,0.0513,0.0681,0.2028,-0.0632,-0.1051,-0.055,0.0938,0.952787


In [64]:
test_x_2020_col = ['tract']
test_x_lag3_col = ['tract']
for i in df_x_year_col:
    test_x_2020_col.append(f'{i}_2020')
    test_x_lag3_col.append(f'{i}_n-5')
test_x_2020 = df_2020.loc[:, test_x_2020_col]
test_x_2020.columns = test_x_lag3_col
test_x_2020

,tract,mean_rent_per_n-5,median_income_n-5,population_n-5,white_pop_n-5,black_pop_n-5,native_pop_n-5,asian_pop_n-5,pacific_pop_n-5,male_population_n-5,...,stocks_bonds_n-5,stocks_corp_bonds_n-5,hist_risk_n-5,inflation_n-5,SP500_n-5,Tbill_3month_n-5,Tbond_10year_n-5,baa_corp_bonds_n-5,real_estate3_n-5,employment_rate_n-5
0,01003010500,0.695000,61232.50,5889.0,5021.0,766.0,16.000,6.00,0.000,3215.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.965224
1,01003010600,0.610000,32000.00,4115.0,1477.0,2339.0,19.000,11.25,0.000,1679.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.837479
2,01003010704,1.050833,79797.00,6106.0,5125.0,756.0,0.000,43.00,0.000,2919.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.988605
3,01003010707,1.112500,56228.75,5530.5,4760.0,333.5,20.625,19.50,0.000,2801.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.971445
4,01003010708,1.015556,56228.75,5530.5,4760.0,333.5,20.625,19.50,0.000,2801.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.971445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,49057210518,1.137917,69671.50,4223.5,3785.0,36.0,18.500,38.25,3.375,2122.5,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.966391
11903,49057210600,0.990833,66125.00,7451.0,6705.0,23.0,25.000,79.00,59.000,3797.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.961838
11904,49057210701,1.249167,62946.50,5979.0,5030.0,17.0,45.000,65.00,0.000,2929.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.965305
11905,49057211100,0.987500,71391.00,5622.0,4965.0,64.0,0.000,21.00,0.000,2631.0,...,0.0669,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.954597


In [65]:
final_test = test_x.merge(test_x_2022, how='left', on='tract')
final_test = final_test.merge(test_x_2021, how='left', on='tract')
final_test = final_test.merge(test_x_2020, how='left', on='tract')
final_test.dropna(inplace = True)
final_test.shape

(11905, 135)

In [66]:
test_data = final_test.iloc[:, 1:]

### Use LGB model to predict

In [67]:
LGB = XGBRegressor(learning_rate= 0.1, max_depth= 4, n_estimators= 500)
LGB.fit(X, y)
y_pred = LGB.predict(test_data)
final_testLGB = final_test.copy()
final_testLGB['mean_rent_per_2025'] = y_pred
final_testLGB.head()

,tract,total_units,property_sqft,current_rent,income_tax_rate,corporate_tax_rate,state_sales_tax_rate,avg_local_sales_tax_rate,max_local_sales_tax,avg_household_income,...,stocks_corp_bonds_n-5,hist_risk_n-5,inflation_n-5,SP500_n-5,Tbill_3month_n-5,Tbond_10year_n-5,baa_corp_bonds_n-5,real_estate3_n-5,employment_rate_n-5,mean_rent_per_2025
0,01003010500,68.000000,65200.000000,633.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.965224,0.752621
1,01003010600,56.000000,61800.000000,691.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.837479,0.756096
2,01003010704,180.000000,160288.000000,1417.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.988605,1.742982
3,01003010707,216.000000,207324.000000,1419.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.971445,1.708735
4,01003010708,228.666667,269810.333333,1395.333333,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.971445,1.411977


In [68]:
final_testLGB = final_testLGB.loc[:, ['tract', 'mean_rent_per_2025']]
final_testLGB = final_testLGB.merge(df_y, how='left', on='tract')
final_testLGB.sort_values(by = 'mean_rent_per_2025', ascending = False).reset_index(drop=True)

,tract,mean_rent_per_2025,mean_rent_per_2022
0,06037273403,7.920712,7.866
1,12099003512,7.493563,7.994
2,06073008202,7.175119,7.092
3,06037570403,6.887536,6.316
4,06073007304,6.726163,5.684
...,...,...,...
11900,05035030101,0.540480,0.536
11901,01095030401,0.532173,0.450
11902,13215003302,0.518160,0.466
11903,01103005303,0.503693,0.482


In [69]:
final_testLGB['rent_growth_2025'] = (final_testLGB['mean_rent_per_2025']-
                                    final_testLGB['mean_rent_per_2022'])/final_testLGB['mean_rent_per_2022']
final_testLGB = final_testLGB.sort_values(by = 'rent_growth_2025', ascending = False).reset_index(drop=True)
final_testLGB = final_testLGB.loc[:, ['tract', 'rent_growth_2025']]

In [70]:
final_testLGB.rename(columns={"rent_growth_2025": "LGB_rent_growth_2025"}, inplace = True)

In [71]:
final_testLGB

,tract,LGB_rent_growth_2025
0,18043070801,0.605929
1,04013815401,0.530372
2,06037800205,0.452792
3,13021011100,0.423584
4,40131050105,0.380768
...,...,...
11900,06037214700,-0.078537
11901,12086001605,-0.080395
11902,48135002502,-0.080497
11903,48309003200,-0.092510


### Use XGB model to predict

In [72]:
XG = XGBRegressor(learning_rate=0.1, max_depth=4, n_estimators=800)
XG.fit(X, y)
y_pred = XG.predict(test_data)
final_testXG = final_test.copy()
final_testXG['mean_rent_per_2025'] = y_pred
final_testXG.head()

,tract,total_units,property_sqft,current_rent,income_tax_rate,corporate_tax_rate,state_sales_tax_rate,avg_local_sales_tax_rate,max_local_sales_tax,avg_household_income,...,stocks_corp_bonds_n-5,hist_risk_n-5,inflation_n-5,SP500_n-5,Tbill_3month_n-5,Tbond_10year_n-5,baa_corp_bonds_n-5,real_estate3_n-5,employment_rate_n-5,mean_rent_per_2025
0,01003010500,68.000000,65200.000000,633.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.965224,0.749090
1,01003010600,56.000000,61800.000000,691.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.837479,0.748505
2,01003010704,180.000000,160288.000000,1417.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.988605,1.760354
3,01003010707,216.000000,207324.000000,1419.000000,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.971445,1.704372
4,01003010708,228.666667,269810.333333,1395.333333,0.036667,0.065,0.04,0.052386,0.075,80251.0,...,0.0761,0.0484,0.0136,0.1644,-0.0125,0.0984,0.0893,0.0886,0.971445,1.395611


In [73]:
final_testXG = final_testXG.loc[:, ['tract', 'mean_rent_per_2025']]
final_testXG = final_testXG.merge(df_y, how='left', on='tract')
final_testXG.sort_values(by = 'mean_rent_per_2025', ascending = False).reset_index(drop=True)
final_testXG

,tract,mean_rent_per_2025,mean_rent_per_2022
0,01003010500,0.749090,0.694000
1,01003010600,0.748505,0.670000
2,01003010704,1.760354,1.424000
3,01003010707,1.704372,1.428000
4,01003010708,1.395611,1.213333
...,...,...,...
11900,49057210518,1.658476,1.426000
11901,49057210600,1.442916,1.220000
11902,49057210701,1.893485,1.525000
11903,49057211100,1.540080,1.366000


In [74]:
final_testXG['rent_growth_2025'] = (final_testXG['mean_rent_per_2025']-
                                    final_testXG['mean_rent_per_2022'])/final_testXG['mean_rent_per_2022']
final_testXG = final_testXG.sort_values(by = 'rent_growth_2025', ascending = False).reset_index(drop=True)
final_testXG

,tract,mean_rent_per_2025,mean_rent_per_2022,rent_growth_2025
0,04013815401,2.707880,1.726000,0.568876
1,18043070801,1.336220,0.856000,0.561004
2,06037800205,6.709728,4.576000,0.466287
3,13021011100,1.069154,0.745000,0.435106
4,40131050105,0.923545,0.664000,0.390880
...,...,...,...,...
11900,12086001605,2.001041,2.172000,-0.078710
11901,48135002502,0.911946,0.992000,-0.080700
11902,48309003200,1.361002,1.500000,-0.092665
11903,12097040811,1.931351,2.145333,-0.099743


In [75]:
final_testXG.rename(columns={"rent_growth_2025": "XG_rent_growth_2025"}, inplace = True)
final_testXG

,tract,mean_rent_per_2025,mean_rent_per_2022,XG_rent_growth_2025
0,04013815401,2.707880,1.726000,0.568876
1,18043070801,1.336220,0.856000,0.561004
2,06037800205,6.709728,4.576000,0.466287
3,13021011100,1.069154,0.745000,0.435106
4,40131050105,0.923545,0.664000,0.390880
...,...,...,...,...
11900,12086001605,2.001041,2.172000,-0.078710
11901,48135002502,0.911946,0.992000,-0.080700
11902,48309003200,1.361002,1.500000,-0.092665
11903,12097040811,1.931351,2.145333,-0.099743


### Calculate ROI

In [76]:
final_testXG['ROI'] = ((((1+final_testXG['XG_rent_growth_2025'])*final_testXG['mean_rent_per_2022'])-
                       (final_testXG['mean_rent_per_2022']/2))/.04)/(final_testXG['mean_rent_per_2022']/0.04)
final_testXG

,tract,mean_rent_per_2025,mean_rent_per_2022,XG_rent_growth_2025,ROI
0,04013815401,2.707880,1.726000,0.568876,1.068876
1,18043070801,1.336220,0.856000,0.561004,1.061004
2,06037800205,6.709728,4.576000,0.466287,0.966287
3,13021011100,1.069154,0.745000,0.435106,0.935106
4,40131050105,0.923545,0.664000,0.390880,0.890880
...,...,...,...,...,...
11900,12086001605,2.001041,2.172000,-0.078710,0.421290
11901,48135002502,0.911946,0.992000,-0.080700,0.419300
11902,48309003200,1.361002,1.500000,-0.092665,0.407335
11903,12097040811,1.931351,2.145333,-0.099743,0.400257


In [77]:
sumROI = 0
for i in range(0, 10):
    sumROI += final_testXG['ROI'][i]
sumROI

9.172998557669526

In [78]:
final_testXG

,tract,mean_rent_per_2025,mean_rent_per_2022,XG_rent_growth_2025,ROI
0,04013815401,2.707880,1.726000,0.568876,1.068876
1,18043070801,1.336220,0.856000,0.561004,1.061004
2,06037800205,6.709728,4.576000,0.466287,0.966287
3,13021011100,1.069154,0.745000,0.435106,0.935106
4,40131050105,0.923545,0.664000,0.390880,0.890880
...,...,...,...,...,...
11900,12086001605,2.001041,2.172000,-0.078710,0.421290
11901,48135002502,0.911946,0.992000,-0.080700,0.419300
11902,48309003200,1.361002,1.500000,-0.092665,0.407335
11903,12097040811,1.931351,2.145333,-0.099743,0.400257


## Compare prediction results from different models

### Import Fixed Effect prediction result

In [79]:
final_testFE = pd.read_csv('Final_fixed_data_needed.csv', index_col = 0)
final_testFE.rename(columns={"growth_25": "FE_rent_growth_2025"}, inplace = True)
final_testFE.head()

,tract,FE_rent_growth_2025
0,1003010500,-0.175001
1,1003010600,0.001785
2,1003010704,0.079321
3,1003010707,0.048250
4,1003010708,-0.050301


In [80]:
final_testFE['tract']=final_testFE['tract'].apply(lambda x: str(x).zfill(11))
final_testFE

,tract,FE_rent_growth_2025
0,01003010500,-0.175001
1,01003010600,0.001785
2,01003010704,0.079321
3,01003010707,0.048250
4,01003010708,-0.050301
...,...,...
11760,49057210518,0.048809
11761,49057210600,-0.001031
11762,49057210701,0.055992
11763,49057211100,0.114976


In [81]:
final = final_testFE.merge(final_testXG, how='left', on='tract')
final = final.merge(final_testLGB, how='left', on='tract')

In [82]:
final = final[['tract', 'FE_rent_growth_2025', 'XG_rent_growth_2025', 'LGB_rent_growth_2025']] 
final

,tract,FE_rent_growth_2025,XG_rent_growth_2025,LGB_rent_growth_2025
0,01003010500,-0.175001,0.079381,0.084468
1,01003010600,0.001785,0.117171,0.128502
2,01003010704,0.079321,0.236204,0.224004
3,01003010707,0.048250,0.193538,0.196593
4,01003010708,-0.050301,0.150229,0.163717
...,...,...,...,...
11760,49057210518,0.048809,0.163026,0.177822
11761,49057210600,-0.001031,0.182718,0.201666
11762,49057210701,0.055992,0.241629,0.243701
11763,49057211100,0.114976,0.127438,0.134253


In [83]:
final.sort_values(by = 'FE_rent_growth_2025', ascending = False).reset_index(drop=True).head(10)

,tract,FE_rent_growth_2025,XG_rent_growth_2025,LGB_rent_growth_2025
0,04013217001,1.684731,0.161208,0.171603
1,06059062645,1.525346,0.136040,0.141538
2,06059062658,1.473987,0.120287,0.123380
3,06037264000,1.325111,0.066382,0.054277
4,48201333204,1.297981,0.083040,0.071199
5,18043070801,1.276547,0.561004,0.605929
6,06059063010,1.217841,0.142234,0.144315
7,06059062649,1.181896,0.100328,0.108008
8,48201421402,1.153177,0.050812,0.068860
9,22051020517,1.034263,0.095851,0.103653


In [84]:
final.sort_values(by = 'XG_rent_growth_2025', ascending = False).reset_index(drop=True).head(10)

,tract,FE_rent_growth_2025,XG_rent_growth_2025,LGB_rent_growth_2025
0,18043070801,1.276547,0.561004,0.605929
1,06037800205,0.586632,0.466287,0.452792
2,13021011100,0.369543,0.435106,0.423584
3,40131050105,0.097562,0.390880,0.380768
4,37119005200,-0.255410,0.368388,0.349797
5,22113950600,-0.153783,0.352597,0.374065
6,40109102500,-0.134369,0.344746,0.340528
7,12081002025,0.192506,0.342930,0.330297
8,13135050747,0.019494,0.342184,0.329722
9,40109101300,0.009879,0.339300,0.354523


In [85]:
final.sort_values(by = 'LGB_rent_growth_2025', ascending = False).reset_index(drop=True).head(10)

,tract,FE_rent_growth_2025,XG_rent_growth_2025,LGB_rent_growth_2025
0,18043070801,1.276547,0.561004,0.605929
1,06037800205,0.586632,0.466287,0.452792
2,13021011100,0.369543,0.435106,0.423584
3,40131050105,0.097562,0.390880,0.380768
4,04013050620,0.364423,0.326314,0.379444
5,22113950600,-0.153783,0.352597,0.374065
6,12091020800,-0.213282,0.328451,0.367889
7,40109101300,0.009879,0.339300,0.354523
8,37119005200,-0.255410,0.368388,0.349797
9,40109102500,-0.134369,0.344746,0.340528


In [86]:
prediction_final = final_testXG[['tract', 'XG_rent_growth_2025']].sort_values(by = 'XG_rent_growth_2025', ascending = False).reset_index(drop=True)
prediction_final

,tract,XG_rent_growth_2025
0,04013815401,0.568876
1,18043070801,0.561004
2,06037800205,0.466287
3,13021011100,0.435106
4,40131050105,0.390880
...,...,...
11900,12086001605,-0.078710
11901,48135002502,-0.080700
11902,48309003200,-0.092665
11903,12097040811,-0.099743


In [87]:
prediction_final['ROI'] = final_testXG['ROI']
prediction_final

,tract,XG_rent_growth_2025,ROI
0,04013815401,0.568876,1.068876
1,18043070801,0.561004,1.061004
2,06037800205,0.466287,0.966287
3,13021011100,0.435106,0.935106
4,40131050105,0.390880,0.890880
...,...,...,...
11900,12086001605,-0.078710,0.421290
11901,48135002502,-0.080700,0.419300
11902,48309003200,-0.092665,0.407335
11903,12097040811,-0.099743,0.400257


In [88]:
prediction_final.to_csv('Prediction.csv')

## Create dataframe to draw Choropleth Maps

In [89]:
for_map = final.sort_values(by = 'XG_rent_growth_2025', ascending = False).reset_index(drop=True)
for_map

,tract,FE_rent_growth_2025,XG_rent_growth_2025,LGB_rent_growth_2025
0,18043070801,1.276547,0.561004,0.605929
1,06037800205,0.586632,0.466287,0.452792
2,13021011100,0.369543,0.435106,0.423584
3,40131050105,0.097562,0.390880,0.380768
4,37119005200,-0.255410,0.368388,0.349797
...,...,...,...,...
11760,12086001605,-0.544775,-0.078710,-0.080395
11761,48135002502,-0.430307,-0.080700,-0.080497
11762,48309003200,-0.147288,-0.092665,-0.092510
11763,12097040811,-0.233020,-0.099743,-0.077570


In [90]:
for_map['county'] = for_map.apply(lambda row: row.tract[0: 5], axis = 1)
for_map

,tract,FE_rent_growth_2025,XG_rent_growth_2025,LGB_rent_growth_2025,county
0,18043070801,1.276547,0.561004,0.605929,18043
1,06037800205,0.586632,0.466287,0.452792,06037
2,13021011100,0.369543,0.435106,0.423584,13021
3,40131050105,0.097562,0.390880,0.380768,40131
4,37119005200,-0.255410,0.368388,0.349797,37119
...,...,...,...,...,...
11760,12086001605,-0.544775,-0.078710,-0.080395,12086
11761,48135002502,-0.430307,-0.080700,-0.080497,48135
11762,48309003200,-0.147288,-0.092665,-0.092510,48309
11763,12097040811,-0.233020,-0.099743,-0.077570,12097


In [91]:
for_map.value_counts('county')

county
06037    953
48201    707
04013    489
48113    430
06073    362
        ... 
37167      1
37141      1
13207      1
13225      1
28137      1
Length: 296, dtype: int64

In [92]:
len(for_map['county'].unique())

296

In [93]:
for_map = for_map.groupby('county')[['XG_rent_growth_2025']].median()
for_map

,XG_rent_growth_2025
county,
01003,0.164204
01007,0.097236
01017,0.104728
01021,0.127029
01071,0.056140
...,...
49035,0.159775
49043,0.123754
49045,0.134973


In [94]:
for_map = for_map.reset_index()
for_map

,county,XG_rent_growth_2025
0,01003,0.164204
1,01007,0.097236
2,01017,0.104728
3,01021,0.127029
4,01071,0.056140
...,...,...
291,49035,0.159775
292,49043,0.123754
293,49045,0.134973
294,49049,0.141252


In [ ]:
#for_map.to_csv('growth_formap.csv')